## **データ整理**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from tqdm import tqdm

from myUtils import reduce_mem_usage

import gc

#### [参考]submissionファイルに合わせてaggurigationした学習データ作成
https://www.kaggle.com/sachina/convert-accuracy-to-uncertainty-poisson

In [2]:
quants = ['0.005', '0.025', '0.165', '0.250', '0.500', '0.750', '0.835', '0.975', '0.995']
days = range(1, 1914)
val_eval = ['validation', 'evaluation']
#time_series_columns = [f'd_{i}' for i in days]

def CreateSales( train_sales,name_list, group):
    '''This function returns a dataframe (sales) on the aggregation level given by name list and group'''
    
    rows_ve = [(name + "_X_" + str(q) + "_" + ve, str(q)) for name in name_list for q in quants for ve in val_eval]
    time_series_columns = [c for c in train_sales.columns if "d_" in c]
    sales = train_sales.groupby(group)[time_series_columns].sum() #would not be necessary for lowest level
    return sales


def createTrainSet(sales_train_s,train_sales, name, group_level, X = False):
    sales_total = CreateSales(train_sales,name, group_level)
    if(X == True):
        sales_total = sales_total.rename(index = lambda s:  s + '_X')
    sales_train_s = sales_train_s.append(sales_total)
    return(sales_train_s)

def get_agg_df(train_sales):
    total = ['Total']
    train_sales['Total'] = 'Total'
    train_sales['state_cat'] = train_sales.state_id + "_" + train_sales.cat_id
    train_sales['state_dept'] = train_sales.state_id + "_" + train_sales.dept_id
    train_sales['store_cat'] = train_sales.store_id + "_" + train_sales.cat_id
    train_sales['store_dept'] = train_sales.store_id + "_" + train_sales.dept_id
    train_sales['state_item'] = train_sales.state_id + "_" + train_sales.item_id
    train_sales['item_store'] = train_sales.item_id + "_" + train_sales.store_id
    total = ['Total']
    states = ['CA', 'TX', 'WI']
    num_stores = [('CA',4), ('TX',3), ('WI',3)]
    stores = [x[0] + "_" + str(y + 1) for x in num_stores for y in range(x[1])]
    cats = ['FOODS', 'HOBBIES', 'HOUSEHOLD']
    num_depts = [('FOODS',3), ('HOBBIES',2), ('HOUSEHOLD',2)]
    depts = [x[0] + "_" + str(y + 1) for x in num_depts for y in range(x[1])]
    state_cats = [state + "_" + cat for state in states for cat in cats]
    state_depts = [state + "_" + dept for state in states for dept in depts]
    store_cats = [store + "_" + cat for store in stores for cat in cats]
    store_depts = [store + "_" + dept for store in stores for dept in depts]
    prods = list(train_sales.item_id.unique())
    prod_state = [prod + "_" + state for prod in prods for state in states]
    prod_store = [prod + "_" + store for prod in prods for store in stores]
    cols = [i for i in train_sales.columns if i.startswith('F')]
    sales_train_s = train_sales[cols]
    sales_train_s = pd.DataFrame()
    sales_train_s = createTrainSet(sales_train_s,train_sales, total, 'Total', X=True) #1
    sales_train_s = createTrainSet(sales_train_s, train_sales,states, 'state_id', X=True) #2
    sales_train_s = createTrainSet(sales_train_s,train_sales, stores, 'store_id', X=True) #3
    sales_train_s = createTrainSet(sales_train_s,train_sales, cats, 'cat_id', X=True) #4
    sales_train_s = createTrainSet(sales_train_s,train_sales, depts, 'dept_id', X=True) #5
    sales_train_s = createTrainSet(sales_train_s,train_sales, state_cats, 'state_cat') #6
    sales_train_s = createTrainSet(sales_train_s,train_sales, state_depts, 'state_dept') #7
    sales_train_s = createTrainSet(sales_train_s,train_sales, store_cats, 'store_cat') #8
    sales_train_s = createTrainSet(sales_train_s,train_sales, store_depts, 'store_dept') #9
    sales_train_s = createTrainSet(sales_train_s,train_sales, prods, 'item_id', X=True) #10
    sales_train_s = createTrainSet(sales_train_s,train_sales, prod_state, 'state_item') #11
    sales_train_s = createTrainSet(sales_train_s,train_sales, prod_store, 'item_store')
    sales_train_s['id'] = sales_train_s.index
    return(sales_train_s)

In [3]:
sales_train_val = pd.read_csv(os.path.join("rawdata","sales_train_evaluation.csv")) # データの読み出し
agg_df = get_agg_df(sales_train_val)
agg_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,id
Total_X,32631,31749,23783,25412,19146,29211,28010,37932,32736,25572,...,54308,59921,42362,38777,37096,36963,42552,51518,54338,Total_X
CA_X,14195,13805,10108,11047,9925,11322,12251,16610,14696,11822,...,23037,24704,17721,16150,15678,16297,17430,23103,24644,CA_X
TX_X,9438,9630,6778,7381,5912,9006,6226,9440,9376,7319,...,12366,16249,12228,11370,10375,9162,12303,13681,14815,TX_X
WI_X,8998,8314,6897,6984,3309,8883,9533,11882,8664,6431,...,18905,18968,12413,11257,11043,11504,12819,14734,14879,WI_X
CA_1_X,4337,4155,2816,3051,2630,3276,3450,5437,4340,3157,...,6245,6707,4568,3949,3995,4136,4433,5764,6289,CA_1_X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_516_TX_2,0,1,0,0,0,0,1,1,0,0,...,0,0,1,0,0,0,0,0,1,HOUSEHOLD_2_516_TX_2
HOUSEHOLD_2_516_TX_3,2,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,2,HOUSEHOLD_2_516_TX_3
HOUSEHOLD_2_516_WI_1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,HOUSEHOLD_2_516_WI_1
HOUSEHOLD_2_516_WI_2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,HOUSEHOLD_2_516_WI_2


In [4]:
agg_df.to_pickle(os.path.join("mydata","agg_train_eval.pickle"))